Please note that you need to have a running Spark cluster to execute these commands!

Create a SparkSession.

In [13]:
!pip install pyspark

In [14]:
import pyspark

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

In [16]:
spark

Create a DataFrame from a CSV file.

In [17]:
df = spark.read.option('header','true').csv('heart.csv')

In [18]:
# tell pyspark the type of the columns - saves time on large dataset. there are other ways to do this, but that's my favorite
schema = 'Age INTEGER, Sex STRING, ChestPainType STRING'
df = spark.read.csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/data-engineering/heart.csv', schema=schema, header=True)

In [19]:
# let PySpark infer the schema
df = spark.read.csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/data-engineering/heart.csv', inferSchema=True, header=True)

In [20]:
# replace nulls with other value at reading time
df = spark.read.csv('C:/Users/lynst/Documents/GitHub/machine-learning-projects/data-engineering/heart.csv', nullValue='NA')

In [21]:
# save data
df.write.format("csv").save("heart_save.csv")

AnalysisException: path file:/c:/Users/lynst/Documents/GitHub/machine-learning-projects/data-engineering/heart_save.csv already exists.

Saving won't let you write over an existing file. To do that, you need to set the 'mode' to overwrite:

In [22]:
# if you want to overwrite the file
df.write.format("csv").mode("overwrite").save("heart_save.csv")

Py4JJavaError: An error occurred while calling o103.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	

In [23]:
# show head of table
df.show(3)

+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|_c0|_c1|          _c2|      _c3|        _c4|      _c5|       _c6|  _c7|           _c8|    _c9|    _c10|        _c11|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
|Age|Sex|ChestPainType|RestingBP|Cholesterol|FastingBS|RestingECG|MaxHR|ExerciseAngina|Oldpeak|ST_Slope|HeartDisease|
| 40|  M|          ATA|      140|        289|        0|    Normal|  172|             N|      0|      Up|           0|
| 49|  F|          NAP|      160|        180|        0|    Normal|  156|             N|      1|    Flat|           1|
+---+---+-------------+---------+-----------+---------+----------+-----+--------------+-------+--------+------------+
only showing top 3 rows



In [24]:
# count number of rows
df.count()

919

In [25]:
# show parts of the table
df.select('Age').show(3)
df.select(['Age', 'Sex']).show(3)

AnalysisException: Column 'Age' does not exist. Did you mean one of the following? [_c0, _c1, _c2, _c3, _c4, _c5, _c6, _c7, _c8, _c9, _c10, _c11];
'Project ['Age]
+- Relation [_c0#361,_c1#362,_c2#363,_c3#364,_c4#365,_c5#366,_c6#367,_c7#368,_c8#369,_c9#370,_c10#371,_c11#372] csv


## Caching
every time you run a DAG, it will be re-computed from the beginning. that is, the results are not saved in memory. so, if we want to save a result so it won't have to be recomputed, we can use the cache command. note, that this will occupy space in the working node's memory - so be careful with the sizes of datasets you are caching! by default, the cached DF is stored to RAM, and is unserialized (not converted into a stream of bytes). you can change both of these - store data to hard disk, serialized it, or both!

## Collecting
even after caching a DataFrame, it still sits in the worker nodes memory. if you want to collect is pieces, assemble them and save them on the master node so you won't have to pull it every time, use the command for collecting. again, be very careful with this, since the collected file will have to fit in the master node memory!

In [ ]:
df.cache()
df.collect()

In [ ]:
# convert PySpark DataFrame to Pandas DataFrame
pd_df = df.toPandas()
# convert it back
spark_df = spark.createDataFrame(pd_df)

In [ ]:
# show first three rows as three row objects, which is how spark represents single rows from a table.
# we will learn more about it later
df.head(3)

Print the DataFrame's schema.

In [ ]:
# type as columns
df.printSchema()

In [ ]:
# column dtypes as list of tuples
df.dtypes

In [ ]:
# cast a column from one type to other
from pyspark.sql.types import FloatType

df = df.withColumn("Age", df.Age.cast(FloatType()))
df = df.withColumn("RestingBP", df.Age.cast(FloatType()))

In [ ]:
# compute summery statistics
df.select(['Age', 'RestingBP']).describe().show()

In [ ]:
# add a new column or replace existing one
AgeFixed = df['Age'] + 1  # select alwayes returns a DataFrame object, and we need a column object
df = df.withColumn('AgeFixed', AgeFixed)

In [ ]:
df.select(['AgeFixed', 'Age']).describe().show()

In [ ]:
# remove columns
df.drop('AgeFixed').show(1)  # add df = to get the new DataFrame into a variable

In [ ]:
# rename a column
df.withColumnRenamed('Age', 'age').select('age').show(1)
# to rename more than a single column, i would suggest a loop.
name_pairs = [('Age', 'age'), ('Sex', 'sex')]
for old_name, new_name in name_pairs:
    df = df.withColumnRenamed(old_name, new_name)

In [ ]:
df.select(['age','sex']).show(1)

In [ ]:
# drop all rows that contain any NA
df = df.na.drop()
df.count()
# drop all rows where all values are NA
df = df.na.drop(how='all')
# drop all rows where more at least 2 values are NOT NA
df = df.na.drop(thresh=2)
# drop all rows where any value at specific columns are NAs.
df = df.na.drop(how='any', subset=['age', 'sex'])  # 'any' is the defult

In [ ]:
# fill missing values in a specific column with a '?'
df = df.na.fill(value='?', subset=['sex'])
# replace NAs with mean of column
from pyspark.ml.feature import Imputer  # In statistics, imputation is the process of
# replacing missing data with substituted values
imptr = Imputer(inputCols=['age', 'RestingBP'],
                outputCols=['age', 'RestingBP']).setStrategy('mean')  # can also be 'median' and so on

df = imptr.fit(df).transform(df)

In [ ]:
# filter to adults only and calculate mean
df.filter('age > 18')
df.where('age > 18')  # 'where' is an alias to 'filter'
df.where(df['age'] > 18)  # third option
# add another condition ('&' means and, '|' means or)
df.where((df['age'] > 18) | (df['ChestPainType'] == 'ATA'))
# take every record where the 'ChestPainType' is NOT 'ATA'
df.filter(~(df['ChestPainType'] == 'ATA'))

In [ ]:
df.filter('age > 18').show()

In [ ]:
# evaluate a string expression into command
from pyspark.sql.functions import expr

exp = 'age + 0.2 * AgeFixed'
df.withColumn('new_col', expr(exp)).select('new_col').show(3)

In [ ]:
# group by age
disease_by_age = df.groupby('age').mean().select(['age', 'avg(HeartDisease)'])
# sort values in desnding order
from pyspark.sql.functions import desc

disease_by_age.orderBy(desc("age")).show(5)

Show the first 10 rows of the DataFrame.

In [ ]:
df.show(10)

Group the DataFrame by a column and compute the average of another column.

In [ ]:
df.groupBy("column1").avg("column2")

Filter the DataFrame based on a condition.

In [ ]:
df.filter(df["column1"] > 5).show()

Stop the SparkSession.

In [ ]:
spark.stop()